In [1]:
import sqlite3
import pandas as pd

In [18]:
con = sqlite3.connect('nba.db')
cur = con.cursor()

In [19]:
attach_db1 = "attach database 'nba_1.db' as db1";
attach_db2 = "attach database 'nba_2.db' as db2";

cur.execute(attach_db1)
cur.execute(attach_db2)

In [20]:
source_table = 'db2.spreads'
target_table = 'spreads'

stmt = "SELECT * FROM %s" % source_table

cur.execute(stmt)
data = cur.fetchall()

fields = ','.join('?' for desc in cur.description)
stmt = "insert into {} values ({})".format(target_table, fields)
cur.executemany(stmt, data)
con.commit()
#connection.close()

In [21]:
spreads_df = pd.read_sql('SELECT * FROM spreads', con)
spreads_df.head()

,SEASON,GM_DATE,HOME_TEAM,AWAY_TEAM,AWAY_SCOREBOARD,HOME_SCOREBOARD,AWAY_SPREAD,HOME_SPREAD
0,2021-22,2021-10-19,Milwaukee,Brooklyn,"25,34,26,19,104","37,29,31,30,127","+1½-110,+2-113,+2-113,+2½-135,-","-1½-110,-2-108,-2-108,-2½-105,-"
1,2021-22,2021-10-19,L.A. Lakers,Golden State,"32,21,30,38,121","34,25,26,29,114","+3-110,+3-112,+3-112,+4-135,-","-3-110,-3-109,-3-109,-4-105,-"
2,2021-22,2021-10-20,Charlotte,Indiana,"38,37,13,34,122","27,32,33,31,123","-1+100,+½-110,+1½-110,-1½-110,-","+1-120,-½-110,-1½-112,+1½-110,-"
3,2021-22,2021-10-20,Detroit,Chicago,"14,26,31,23,94","20,24,25,19,88","-5-110,-5-112,-5-112,-5-110,-","+5-110,+5-110,+5-110,+5-110,-"
4,2021-22,2021-10-20,Toronto,Washington,"26,31,24,17,98","18,19,22,24,83","+2½-110,+2½-109,+3-109,+4½-160,-","-2½-110,-2½-112,-3-112,-4½+115,-"


In [22]:
spreads_df = spreads_df[spreads_df['SEASON']!= '2015-16']
spreads_df['SEASON'].unique()

array(['2021-22', '2020-21', '2017-18', '2019-20', '2018-19'],
      dtype=object)

In [23]:
spreads_df.drop(columns=['SEASON', 'AWAY_TEAM', 'AWAY_SCOREBOARD', 'HOME_SCOREBOARD'], inplace=True)
spreads_df.head()

,GM_DATE,HOME_TEAM,AWAY_SPREAD,HOME_SPREAD
0,2021-10-19,Milwaukee,"+1½-110,+2-113,+2-113,+2½-135,-","-1½-110,-2-108,-2-108,-2½-105,-"
1,2021-10-19,L.A. Lakers,"+3-110,+3-112,+3-112,+4-135,-","-3-110,-3-109,-3-109,-4-105,-"
2,2021-10-20,Charlotte,"-1+100,+½-110,+1½-110,-1½-110,-","+1-120,-½-110,-1½-112,+1½-110,-"
3,2021-10-20,Detroit,"-5-110,-5-112,-5-112,-5-110,-","+5-110,+5-110,+5-110,+5-110,-"
4,2021-10-20,Toronto,"+2½-110,+2½-109,+3-109,+4½-160,-","-2½-110,-2½-112,-3-112,-4½+115,-"


In [24]:
spreads_df[['BOOK_1_AWAY', 'BOOK_2_AWAY', 'BOOK_3_AWAY', 'BOOK_4_AWAY', 'DISCARD_AWAY']] = spreads_df['AWAY_SPREAD'].str.split(pat=',', expand=True)
spreads_df[['BOOK_1_HOME', 'BOOK_2_HOME', 'BOOK_3_HOME', 'BOOK_4_HOME', 'DISCARD_HOME']] = spreads_df['HOME_SPREAD'].str.split(pat=',', expand=True)


spreads_df.drop(columns=['AWAY_SPREAD', 'HOME_SPREAD','DISCARD_AWAY','DISCARD_HOME'], inplace=True)

spreads_df.head()

,GM_DATE,HOME_TEAM,BOOK_1_AWAY,BOOK_2_AWAY,BOOK_3_AWAY,BOOK_4_AWAY,BOOK_1_HOME,BOOK_2_HOME,BOOK_3_HOME,BOOK_4_HOME
0,2021-10-19,Milwaukee,+1½-110,+2-113,+2-113,+2½-135,-1½-110,-2-108,-2-108,-2½-105
1,2021-10-19,L.A. Lakers,+3-110,+3-112,+3-112,+4-135,-3-110,-3-109,-3-109,-4-105
2,2021-10-20,Charlotte,-1+100,+½-110,+1½-110,-1½-110,+1-120,-½-110,-1½-112,+1½-110
3,2021-10-20,Detroit,-5-110,-5-112,-5-112,-5-110,+5-110,+5-110,+5-110,+5-110
4,2021-10-20,Toronto,+2½-110,+2½-109,+3-109,+4½-160,-2½-110,-2½-112,-3-112,-4½+115


In [25]:
spreads_df['SPREAD_1_AWAY'] = spreads_df['BOOK_1_AWAY'].astype(str).str[:-4]
spreads_df['ODDS_1_AWAY'] = spreads_df['BOOK_1_AWAY'].astype(str).str[-4:]
spreads_df['SPREAD_2_AWAY'] = spreads_df['BOOK_2_AWAY'].astype(str).str[:-4]
spreads_df['ODDS_2_AWAY'] = spreads_df['BOOK_2_AWAY'].astype(str).str[-4:]
spreads_df['SPREAD_3_AWAY'] = spreads_df['BOOK_3_AWAY'].astype(str).str[:-4]
spreads_df['ODDS_3_AWAY'] = spreads_df['BOOK_3_AWAY'].astype(str).str[-4:]
spreads_df['SPREAD_4_AWAY'] = spreads_df['BOOK_4_AWAY'].astype(str).str[:-4]
spreads_df['ODDS_4_AWAY'] = spreads_df['BOOK_4_AWAY'].astype(str).str[-4:]

spreads_df['SPREAD_1_HOME'] = spreads_df['BOOK_1_HOME'].astype(str).str[:-4]
spreads_df['ODDS_1_HOME'] = spreads_df['BOOK_1_HOME'].astype(str).str[-4:]
spreads_df['SPREAD_2_HOME'] = spreads_df['BOOK_2_HOME'].astype(str).str[:-4]
spreads_df['ODDS_2_HOME'] = spreads_df['BOOK_2_HOME'].astype(str).str[-4:]
spreads_df['SPREAD_3_HOME'] = spreads_df['BOOK_3_HOME'].astype(str).str[:-4]
spreads_df['ODDS_3_HOME'] = spreads_df['BOOK_3_HOME'].astype(str).str[-4:]
spreads_df['SPREAD_4_HOME'] = spreads_df['BOOK_4_HOME'].astype(str).str[:-4]
spreads_df['ODDS_4_HOME'] = spreads_df['BOOK_4_HOME'].astype(str).str[-4:]

spreads_df.drop(columns=['BOOK_1_AWAY','BOOK_2_AWAY','BOOK_3_AWAY','BOOK_4_AWAY','BOOK_1_HOME','BOOK_2_HOME','BOOK_3_HOME','BOOK_4_HOME'], inplace=True)
spreads_df.head()

,GM_DATE,HOME_TEAM,SPREAD_1_AWAY,ODDS_1_AWAY,SPREAD_2_AWAY,ODDS_2_AWAY,SPREAD_3_AWAY,ODDS_3_AWAY,SPREAD_4_AWAY,ODDS_4_AWAY,SPREAD_1_HOME,ODDS_1_HOME,SPREAD_2_HOME,ODDS_2_HOME,SPREAD_3_HOME,ODDS_3_HOME,SPREAD_4_HOME,ODDS_4_HOME
0,2021-10-19,Milwaukee,+1½,-110,+2,-113,+2,-113,+2½,-135,-1½,-110,-2,-108,-2,-108,-2½,-105
1,2021-10-19,L.A. Lakers,+3,-110,+3,-112,+3,-112,+4,-135,-3,-110,-3,-109,-3,-109,-4,-105
2,2021-10-20,Charlotte,-1,+100,+½,-110,+1½,-110,-1½,-110,+1,-120,-½,-110,-1½,-112,+1½,-110
3,2021-10-20,Detroit,-5,-110,-5,-112,-5,-112,-5,-110,+5,-110,+5,-110,+5,-110,+5,-110
4,2021-10-20,Toronto,+2½,-110,+2½,-109,+3,-109,+4½,-160,-2½,-110,-2½,-112,-3,-112,-4½,+115


In [26]:
spreads_df.drop_duplicates(inplace=True)

In [27]:
spreads_df.to_csv('spreads_df.csv')